In [5]:
%matplotlib inline
import os.path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle, Arc
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.neighbors import KernelDensity
from sklearn.decomposition import NMF
from sklearn.neighbors import NearestNeighbors

In [2]:
currdir = os.getcwd()

datadir = os.path.abspath(os.path.join(currdir, '../../../'))

In [74]:
datafile = 'outputs/nmf/nearest-neighbor/nmf_W_normalized_players.csv'

df = pd.read_csv(os.path.join(datadir, datafile))

In [75]:
df

,player,team,at_rim,wing+key_3pts,midrange+some_paint,left_block+some_paint,corner+some_wing_3pts
0,A. Pasaol,UE,0.47,0.27,0.11,0.00,0.14
1,A. Melecio,DLSU,0.18,0.27,0.20,0.12,0.22
2,R. Subido,UST,0.08,0.41,0.12,0.10,0.29
3,D. Ildefonso,NU,0.31,0.26,0.11,0.09,0.22
4,J. Ahanmisi,ADU,0.04,0.32,0.16,0.36,0.12
...,...,...,...,...,...,...,...
77,F. Jaboneta,UP,0.41,0.00,0.00,0.00,0.59
78,I. Batalier,UST,0.25,0.24,0.14,0.37,0.00
79,C. Vito,UP,0.37,0.30,0.00,0.00,0.33
80,S. Akomo,UST,0.97,0.00,0.03,0.00,0.00


In [76]:
df[df.team=='ADU']

,player,team,at_rim,wing+key_3pts,midrange+some_paint,left_block+some_paint,corner+some_wing_3pts
4,J. Ahanmisi,ADU,0.04,0.32,0.16,0.36,0.12
9,S. Manganti,ADU,0.27,0.15,0.14,0.19,0.25
12,P. Sarr,ADU,0.49,0.00,0.26,0.24,0.01
33,C. Conner,ADU,0.42,0.17,0.14,0.00,0.28
35,J. Espeleta,ADU,0.19,0.17,0.43,0.00,0.21
38,S. Camacho,ADU,0.76,0.00,0.20,0.00,0.04
56,J. Pingoy,ADU,0.01,0.83,0.07,0.00,0.09
57,M. Dyke,ADU,0.51,0.06,0.18,0.25,0.00
64,C. Catapusan,ADU,0.40,0.08,0.06,0.44,0.02
68,V. Magbuhos,ADU,0.92,0.01,0.07,0.00,0.00


In [77]:
weights_admu = df[df.team=='ADMU'][['at_rim', 'wing+key_3pts', 'midrange+some_paint', 'left_block+some_paint', 'corner+some_wing_3pts']].to_numpy()
weights_adu = df[df.team=='ADU'][['at_rim', 'wing+key_3pts', 'midrange+some_paint', 'left_block+some_paint', 'corner+some_wing_3pts']].to_numpy()
weights_dlsu = df[df.team=='DLSU'][['at_rim', 'wing+key_3pts', 'midrange+some_paint', 'left_block+some_paint', 'corner+some_wing_3pts']].to_numpy()
weights_feu = df[df.team=='FEU'][['at_rim', 'wing+key_3pts', 'midrange+some_paint', 'left_block+some_paint', 'corner+some_wing_3pts']].to_numpy()
weights_nu = df[df.team=='NU'][['at_rim', 'wing+key_3pts', 'midrange+some_paint', 'left_block+some_paint', 'corner+some_wing_3pts']].to_numpy()
weights_ue = df[df.team=='UE'][['at_rim', 'wing+key_3pts', 'midrange+some_paint', 'left_block+some_paint', 'corner+some_wing_3pts']].to_numpy()
weights_up = df[df.team=='UP'][['at_rim', 'wing+key_3pts', 'midrange+some_paint', 'left_block+some_paint', 'corner+some_wing_3pts']].to_numpy()
weights_ust = df[df.team=='UST'][['at_rim', 'wing+key_3pts', 'midrange+some_paint', 'left_block+some_paint', 'corner+some_wing_3pts']].to_numpy()

players_admu = df[df.team=='ADMU'][['player', 'at_rim', 'wing+key_3pts', 'midrange+some_paint', 'left_block+some_paint', 'corner+some_wing_3pts']].to_numpy()
players_adu = df[df.team=='ADU'][['player', 'at_rim', 'wing+key_3pts', 'midrange+some_paint', 'left_block+some_paint', 'corner+some_wing_3pts']].to_numpy()
players_dlsu = df[df.team=='DLSU'][['player', 'at_rim', 'wing+key_3pts', 'midrange+some_paint', 'left_block+some_paint', 'corner+some_wing_3pts']].to_numpy()
players_feu = df[df.team=='FEU'][['player', 'at_rim', 'wing+key_3pts', 'midrange+some_paint', 'left_block+some_paint', 'corner+some_wing_3pts']].to_numpy()
players_nu = df[df.team=='NU'][['player', 'at_rim', 'wing+key_3pts', 'midrange+some_paint', 'left_block+some_paint', 'corner+some_wing_3pts']].to_numpy()
players_ue = df[df.team=='UE'][['player', 'at_rim', 'wing+key_3pts', 'midrange+some_paint', 'left_block+some_paint', 'corner+some_wing_3pts']].to_numpy()
players_up = df[df.team=='UP'][['player', 'at_rim', 'wing+key_3pts', 'midrange+some_paint', 'left_block+some_paint', 'corner+some_wing_3pts']].to_numpy()
players_ust = df[df.team=='UST'][['player', 'at_rim', 'wing+key_3pts', 'midrange+some_paint', 'left_block+some_paint', 'corner+some_wing_3pts']].to_numpy()

weights_team = [weights_admu, weights_adu, weights_dlsu, weights_feu, weights_nu, weights_ue, weights_up, weights_ust]

# weights_np = df[['at_rim', 'wing+key_3pts', 'midrange+some_paint', 'left_block+some_paint', 'corner+some_wing_3pts']].to_numpy()

In [78]:
teams = ['ADMU', 'ADU', 'DLSU', 'FEU', 'NU', 'UE', 'UP', 'UST']

for t in teams:
    df[df.team==t].to_csv('players_{}.csv'.format(t))

In [79]:
nn = NearestNeighbors(n_neighbors=20, metric='minkowski')

In [80]:
player_weights_team = []
player_indices_team = []

for t in weights_team:
    nn.fit(t)
    for i in range(len(t)):
        nns = nn.kneighbors([t[i]], len(t))
        player_weights_team.append(nns[0][0])
        player_indices_team.append(nns[1][0])

pd.DataFrame(player_weights_team).to_csv('nn_weights_teammates.csv')
pd.DataFrame(player_indices_team).to_csv('nn_indices_teammates.csv')
        
# nn.fit(weights_admu)
# nn.kneighbors(weights_admu, len(weights_admu)-1)